# Using HIP on Setonix

## Access to Setonix

Firstly you need a username and password to access Setonix. Your **username** and **password** will be given to you prior to the beginning of this workshop. If you are using your regular Pawsey account then you can reset your password [here](https://support.pawsey.org.au/password-reset/).

Access to Setonix is via Secure SHell (SSH). On Linux, Mac OS, and Windows 10 and higher, an SSH client is available from the command line or terminal application. Otherwise you need to use a client program like [Putty](https://www.putty.org/) or [MobaXterm](https://mobaxterm.mobatek.net/download-home-edition.html).

### Access with SSH on the command line

On the command line use the command **ssh** to access Setonix.

```bash
ssh -Y <username>@setonix.pawsey.org.au
```

#### Passwordless login with SSH

In order to avoid specifying a username and password on each login you can generate a key and password combination on your computer using the following on the command line.

```bash
ssh-keygen -t rsa
```

Then copy the public key (the file that ends in \*.pub) to your account on setonix and append it to the authorized keys in .ssh. On your machine run this command

```bash
scp -r <filename>.pub <username>@setonix.pawsey.org.au
```

Then login to Setonix and run this command

```bash
mkdir -p ${HOME}/.ssh
cat <filename>.pub >> ${HOME}/.ssh/authorized_keys
chmod -R 0400 ${HOME}/.ssh
```

Finally, if you are using MacOS or Linux you can add this line to ${HOME}/.ssh/config on your computer

```text
Host setonix
    Hostname setonix.pawsey.org.au
    IdentityFile <private_key_file>
    User <username>
    ForwardX11 yes
    ForwardAgent yes
    ServerAliveInterval 300
    ServerAliveCountMax 2
    TCPKeepAlive no
```

Then you can run 

```bash
ssh setonix
```

without a password.

### Access from Windows with the MobaXterm client

If you have a OS that is older than Windows 10 and need a client in a hurry, just download **MobaXterm Home (Portable Edition)** from [this location](https://mobaxterm.mobatek.net/download-home-edition.html). Extract the Zip file and run the application. You might need to accept a firewall notification. 

Now go to **Settings -> SSH** and uncheck **"Enable graphical SSH-browser"** in the SSH-browser settings pane. Also enable **"SSH keepalive"** to keep SSH connections active.

<figure style="margin-bottom 3em; margin-top: 2em; margin-left:auto; margin-right:auto; width:100%">
    <img style="vertical-align:middle" src="../images/MobaXTerm_Settings.svg"> <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">Figure: MobaXTerm settings.</figcaption>
</figure>

Then close the settings and start a local terminal.



## Hardware environment

<figure style="margin: 1em; margin-left:auto; margin-right:auto; width:100%;">
    <img src="../images/MI250x.png">
    <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">AMD Instinct<span>&trade;</span> MI250X compute architecture. Image credit: <a href="https://hc34.hotchips.org/")>AMD Instinct<span>&trade;</span> MI200 Series Accelerator and Node Architectures | Hot Chips 34</a></figcaption>
</figure>


### Setonix CPU specifications

| Computer | CPU | Nominal clock frequency (GHz) | Cores | Hardware threads | L1 Cache (KB) | L2 Cache (KB) | L3 cache (MB) | FP SIMD width (bits) | Tflops (FP32 calculated) |
|:----:|:----:|-----:| -----: | -----: | :----: | :----: | :----: | :----: | :----: |
| Setonix |AMD EPYC 7A53 | 2.0 | 64 | 128 | 64x32 | 64x512 | 8x32 | 256 | ~2 |

### Setonix GPU specifications

| Card | Boost clock (GHz)| Compute Units | FP32 Processing Elements | FP64 Processing Elements (equivalent compute capacity) | L1 Cache (KB) | L2 Cache (KB) | device memory (GB) | Peak Tflops (FP32)| Peak Tflops (FP64)|
|:----:|:-----| :----- | :----- | :---- | :---- | :---- | :---- | :---- | :---- |
| AMD Radeon Instinct MI250x |1.7 | 220 | 14080 | 14080 | 220x16 | 16000 | 128 | 47.9 | 47.9 |


## Job queues

On Setonix the following queues are available for general use:

|Queue| Max time limit| Processing elements (CPU) | Socket| Cores| processing elements per CPU core | Available memory (GB) | Number of HIP devices | Memory per HIP device (GB) |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| work | 24 hours | 256 | 2 | 64 | 2 | 230 | 0 | 0 |
| long | 96 hours | 256 | 2 | 64 | 2 | 230 | 0 | 0 |
| debug | 1 hour | 256 | 2 | 64 | 2 | 230 | 0 | 0 |
| highmem | 24 hours | 256 | 2 | 64 | 2 | 980 | 0 | 0 |
| copy | 24 hours | 32 | 1 | 64 | 2 | 118 | 0 | 0 |
| gpu | 24 hours | 128 | 1 | 64 | 2 | 128 | 8 (4x2) | 64 |

## Interactive jobs on GPU nodes

```bash
salloc --account ${PAWSEY_PROJECT} --ntasks 1 --mem 4GB --cpus-per-task 1 --time 1:00:00 --gpus-per-task 2 --partition gpu
```

## Building software for Setonix

### Software modules

There are three main programming environments available on Setonix. Each provides C/C++ and Fortran compilers that build software with knowledge of of the MPI libraries available on Setonix. The **PrgEnv-GNU** programming environment uses the GNU compilers, **PrgEnv-aocc** uses the AMD aocc optimising compiler to try and get the best performance from the AMD CPU's on Setonix, and the **PrgEnv-cray** compilers use the compilers from Cray. Use these commands to find which module to load.

| Programming environment | command to use |
| :--- | :--- |
| AMD | ```module avail PrgEnv-aocc``` |
| Cray | ```module avail PrgEnv-cray``` |
| GNU | ```module avail PrgEnv-gnu``` |

When compiling HIP sources you have the choice of either the the ROCM **hipcc** compiler wrapper or the Cray compiler wrapper **CC** from **PrgEnv-cray**. If you use the Cray compiler wrapper you need to swap to the module **PrgEnv-cray** as the GNU programming environment (**PrgEnv-gnu**) is loaded by default. 

```bash
module swap PrgEnv-gnu PrgEnv-cray
```

Then the following compiler wrappers are available for use to compile source files:

| Command | Explanation |
| :--- | :--- |
| cc | C compiler |
| CC | C++ compiler |
| ftn | FORTRAN compiler |

In order to use the GPU-aware MPI library you also need to load the **craype-accel-amd-gfx90a** module, which works in all three programming environments. To see which version to load run this command.

```bash
module avail craype-accel-amd-gfx90a
```

Load the module **craype-accel-amd-gfx90a** then set the environment variable

```bash
export MPICH_GPU_SUPPORT_ENABLED=1
```

Finally, in order to have ROCM software (such as hipcc and rocgdb) and libraries available you need to have the **rocm** module loaded. To see which one to load run this command:

```bash
module avail rocm
```

The **rocm** module is independent of the programming environment module loaded. 

### Compiling software with HIP and MPI support

According to this [documentation](https://docs.amd.com/bundle/HIP-Programming-Guide-v5.0/page/Transitioning_from_CUDA_to_HIP.html) the AMD compiler wrapper **hipcc** can be use for compiling HIP source files and is the suggested linker for program objects. 

#### Compiling and linking with the **hipcc** compiler wrapper

You can use these compiler flags to bring in the MPI headers and make sure **hipcc** compiles kernels for the MI250X GPU's on Setonix.

| Function | flags |
| :--- | :--- |
| Compile | ```-I${MPICH_DIR}/include --offload-arch=gfx90a ``` |
| Link | ```-L${MPICH_DIR}/lib -lmpi ${PE_MPICH_GTL_DIR_amd_gfx90a} ${PE_MPICH_GTL_LIBS_amd_gfx90a}``` |
| Debug (compile and link) | ```-ggdb``` |
| OpenMP (compile and link)| ```-fopenmp``` |

If you want **hipcc** to behave like Cray **CC**, make sure the **PrgEnv-cray** and **craype-accel-amd-gfx90a** modules are also loaded. Then you can add the output of this command,

```bash
$(CC --cray-print-opts=cflags)
```

to the hipcc compile flags, and the output of this command,

```bash
$(CC --cray-print-opts=libs)
```

to the hipcc linker flags.

#### Compiling and linking with the Cray **CC** compiler wrapper 

If you are using the Cray compiler wrapper **CC** you can add these flags to compile and link HIP code for the MI250X GPU's on Setonix. You need to have the **rocm** module loaded.

| Function | flags |
| :--- | :--- |
| Compile | ```-D__HIP_ROCclr__ -D__HIP_ARCH_GFX90A__=1 --offload-arch=gfx90a -x hip``` |
| Link |  |
| Debug (compile and link) | ```-g``` |
| OpenMP (compile and link)| ```-fopenmp``` |

#### Mixing hipcc and Cray compilation

From this [documentation](https://docs.amd.com/bundle/HIP-Programming-Guide-v5.0/page/Transitioning_from_CUDA_to_HIP.html) it is important to note that all code links back to the same C++ standard libraries. The command ```hipconfig --cxx``` generates extra compile flags that might be useful for including in the build process with the Cray wrapper. 

## Exercise: compile and run your first MPI-enabled HIP application

In the files [hello_devices_mpi.cpp](hello_devices_mpi.cpp) and [hello_devices_mpi_onefile.cpp](hello_devices_mpi_onefile.cpp) are files to implement an MPI-enabled HIP application that reports on devices and fills a vector. The difference between the two is that for [hello_devices_mpi.cpp](hello_devices_mpi.cpp) has the kernel located in a separate file [kernels.hip.cpp](kernels.hip.cpp). Your task is to compile these files into two executables, **hello_devices_mpi.exe** and **hello_devices_mpi_onefile.exe**.

### Compilation steps

1. Log into **setonix.pawsey.org.au**.
1. Use **cd** to change directory to your temporary file location in /scratch.
1. Clone the course material from Github if don't already have it.
    <br></br>
    ```git clone git@github.com:pelagos-consulting/HIP_Course.git```
    <br></br>
1. Change directory to **course_material/L2_Using_HIP_On_Setonix**.
1. Get an interactive job on the GPU queue of Setonix with this command:
    <br></br>
    ```salloc --account ${PAWSEY_PROJECT} --ntasks 1 --mem 4GB --cpus-per-task 1 --time 1:00:00 --gpus-per-task 1 --partition gpu```
    <br></br>
1. Load the **rocm** module
    <br></br>
    ```module load rocm```
    <br></br>    
1. Swap out the **PrgEnv-gnu** module for the **PrgEnv-cray** module
    <br></br>
    ```module swap PrgEnv-gnu PrgEnv-cray```
    <br></br>
1. Load the **craype-accel-amd-gfx90a** module
    <br></br>
    ```module load craype-accel-amd-gfx90a```
    
#### Compile the kernel and main program in separate files
1. Compile the kernel file [kernels.hip.cpp](kernels.hip.cpp)
    <br></br>
    ```hipcc -c kernels.hip.cpp --offload-arch=gfx90a -o kernels.o```
    <br></br>
1. Use **CC** to compile the file [hello_devices_mpi.cpp](hello_devices_mpi.cpp). Make sure to include the location of the **hip_helper.hpp** library, located in ../include.
    <br></br>
    ```CC -c -D__HIP_ROCclr__ -D__HIP_ARCH_GFX90A__=1 --offload-arch=gfx90a -I../include -x hip hello_devices_mpi.cpp -o hello_devices_mpi.o```
    <br></br>
1. Use **hipcc** to link the object files together in a way that is aware of the MPI library.
    <br></br>
    ```hipcc kernels.o hello_devices_mpi.o -o hello_devices_mpi.exe -L${MPICH_DIR}/lib -lmpi ${PE_MPICH_GTL_DIR_amd_gfx90a} ${PE_MPICH_GTL_LIBS_amd_gfx90a}```
<br></br>

#### Compile the combined file in one go using **hipcc**

```hipcc -I${MPICH_DIR}/include -I../include --offload-arch=gfx90a hello_devices_mpi_onefile.cpp -o hello_devices_mpi_onefile_hipcc.exe -L${MPICH_DIR}/lib -lmpi ${PE_MPICH_GTL_DIR_amd_gfx90a} ${PE_MPICH_GTL_LIBS_amd_gfx90a}```
<br></br>

#### Compile the combined file in one go using **CC**

```CC -D__HIP_ROCclr__ -D__HIP_ARCH_GFX90A__=1 --offload-arch=gfx90a -I../include -x hip hello_devices_mpi_onefile.cpp -o hello_devices_mpi_onefile_CC.exe```


### Run the compiled code

Using **srun** we can run the executable files. If we don't use **srun** then it will pick up all the GPU's on a node. 

1. ```srun ./hello_devices_mpi.exe```
1. ```srun ./hello_devices_mpi_onefile_hipcc.exe```
1. ```srun ./hello_devices_mpi_onefile_CC.exe```

### Bonus task: Try running these programs with and without **srun** to see what happens.

### The answer

If you get stuck, the example [Makefile](Makefile) contains the above compilation steps. Assuming you loaded the right modules defined above, the make command is run as follows:

```make clean; make```

The script **run_compile.sh** contains the necessary commands to load the appropriate modules and run the **make** command.

```bash
   chmod 700 run_compile.sh
   ./run_compile.sh
```

## Batch jobs on GPU nodes

Pawsey has extensive documentation available for running jobs, at this [site](https://support.pawsey.org.au/documentation/display/US/Running+Jobs+in+Setonix). The suggested job script below will allocate an MPI task for each compute device on a node of Setonix. Then it will allocate 8 OpenMP threads to each MPI task. We can use the helper program [hello_jobstep]( https://code.ornl.gov/olcf/hello_jobstep) Written by Thomas Papatheodore from ORNL to show the node hostname

```bash
#!/bin/bash -l

#SBATCH --account=pawsey0007       # your account
#SBATCH --partition=gpu            # Using the gpu partition
#SBATCH --ntasks=6                 # Total number of tasks
#SBATCH --ntasks-per-node=8        # Set this for 1 mpi task per compute device
#SBATCH --cpus-per-task=16          # How many OpenMP threads per MPI task 
#SBATCH --gpus-per-task=1          # How many HIP compute devices to allocate to a task
#SBATCH --mem=4000M                #Indicate the amount of memory per node when asking for share resources
#SBATCH --time=01:00:00

module swap PrgEnv-gnu PrgEnv-cray
module load craype-accel-amd-gfx90a
module load rocm

export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK   #To define the number of OpenMP threads available per MPI task, in this case it will be 8
export OMP_PLACES=cores     #To bind threads to cores
export OMP_PROC_BIND=close  #To bind (fix) threads (allocating them as close as possible). This option works together with the "places" indicated above, then: allocates threads in closest cores.
 
# Temporal workaround for avoiding Slingshot issues on shared nodes:
export FI_CXI_DEFAULT_VNI=$(od -vAn -N4 -tu < /dev/urandom)

# Bind options to srun plane=4 is the number of NUMA domains per processor (4)
export BIND_OPTIONS="--cpu-bind=ldoms -m plane=4 --gpu-bind=closest"

# Run a job with task placement and $BIND_OPTIONS
srun -N $SLURM_JOB_NUM_NODES -n $SLURM_NTASKS -c $OMP_NUM_THREADS $BIND_OPTIONS  ./hello_jobstep.exe
```

<address>
Written by Dr. Toby Potter of <a href="https://www.pelagos-consulting.com">Pelagos Consulting and Education</a> for the Pawsey Supercomputing Centre
</address>